First of all, I want to apologize for my English, and you will see the reason.

# EDA(数据探索性分析)

This notebook is mainly used to make a overview of the data in this analysis competition. If you like this notebook, please upvote it and make more kaggler see this, and if you have any idea, please tell me, I will update here as soon as possible.(本notebook主要用于对该次分析竞赛的大量数据进行简单概览，以及普及一些关于NCAA相关的知识，毕竟对于国内的同学来说，看NCAA的人应该是少数吧，提前声明，本人只看NBA，不看NCAA，所以可能有些信息有误的地方，望大家评论区指出，感谢)；

## What is NCAA March Madness(什么是NCAA疯狂三月)

First of all, NCAA is the National University Sports Association of the United States. It is an association of hundreds of universities and colleges in the United States. Its main activities are various sports leagues held every year, among which the basketball leagues in the first half of the year and rugby leagues in the second half of the year are the most concerned. March Madness is actually the NCAA tournament. It was founded in 1939 and is one of the largest national sports events in the United States. Because most of the NCAA tournament match are held in March, so we call it March Madness.(首先，NCAA是美国的全国大学体育协会，它是由美国千百所大学院校所参与结盟的一个协会。其主要活动是每年举办的各种体育项目联赛，其中最受关注的是上半年的篮球联赛和下半年的橄榄球联赛。而疯狂三月其实是通称的NCAA锦标赛，即美国大学体育总会(NCAA)一级联赛男篮锦标赛，成立于1939年，是美国最大的全国性体育赛事之一。因为NCAA锦标赛的大部分比赛都在3月进行，俗称疯狂三月)

In short, March Madness is the time when NCAA Basketball tournament in March every year.(简单说，疯狂三月就是每年三月份，NCAA篮球锦标赛集中进行比赛的这段时间)

### Tournament Rule(赛制规则)

1. The first is the "First Four", which consists of the four lowest ranked champion teams in 32 division areas and the four lowest ranked teams in 36 teams selected by NCAA. The winning four teams and the other 60 teams make up the top 64 teams.(首先进行的是“最先四场（First Four）”，由32个分赛区里排名最低的四支冠军队伍和NCAA选拔的36支队伍中排名最低的四支捉对厮杀，胜出的四队和另外的60支队伍组成64强)
2. 64->32->16->8->4->2->Champion

In addition, it should be noted that the tournament is **one-round**, so compared with the seven games in the NBA playoffs, it is more likely to have black horses, which also brings More surprises.(另外要注意，比赛是**一轮制**，因此相比较NBA季后赛的7场，更加容易出现黑马，这也带来了更大的观赏性)

## Data overview and visualization(数据简单概览及可视化)

### Load lib and set path(加载相关库，设置路径)

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

men_folder_path = '/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/'
stage1_folder_path = 'MDataFiles_Stage1/'

### Load section 1 data and merge(加载数据并merge到一个DataFrame)

In [ ]:
def load_section1():
    folder_path = men_folder_path
    prefix = 'M'
    team_section1 = pd.read_csv(folder_path+stage1_folder_path+prefix+'Teams.csv')
    season_section1 = pd.read_csv(folder_path+stage1_folder_path+prefix+'Seasons.csv')
    seed_section1 = pd.read_csv(folder_path+stage1_folder_path+prefix+'NCAATourneySeeds.csv')
    regular_section1 = pd.read_csv(folder_path+stage1_folder_path+prefix+'RegularSeasonCompactResults.csv')
    nacc_section1 = pd.read_csv(folder_path+stage1_folder_path+prefix+'NCAATourneyCompactResults.csv')
    regular_section1['is_regular'] = 1
    nacc_section1['is_regular'] = 0
    regular_detail_section1 = pd.read_csv(folder_path+stage1_folder_path+prefix+'RegularSeasonDetailedResults.csv')
    nacc_detail_section1 = pd.read_csv(folder_path+stage1_folder_path+prefix+'NCAATourneyDetailedResults.csv')
    regular_section1 = regular_section1.merge(regular_detail_section1)
    nacc_section1 = nacc_section1.merge(nacc_detail_section1)
    
    section1 = regular_section1.append(nacc_section1, ignore_index=True).sort_values(by=['Season','DayNum'])
    section1 = section1.merge(seed_section1,left_on=['Season','WTeamID'],right_on=['Season','TeamID']).drop('TeamID',axis=1)
    section1 = section1.merge(seed_section1,left_on=['Season','LTeamID'],right_on=['Season','TeamID'],suffixes=('_W', '_L')).drop('TeamID',axis=1)
    section1 = section1.merge(season_section1,left_on=['Season'],right_on=['Season'])
    section1 = section1.merge(team_section1,left_on=['WTeamID'],right_on=['TeamID']).drop('TeamID',axis=1)
    section1 = section1.merge(team_section1,left_on=['LTeamID'],right_on=['TeamID'],suffixes=('_W', '_L')).drop('TeamID',axis=1)
    section1 = section1.sort_values(by=['Season','DayNum'])
    
    return section1

mSection1 = load_section1()
mSection1

### Count of Champion and second(冠军、亚军的次数可视化)

In [ ]:
final_round = mSection1.query('DayNum == 154')

plt.subplots(figsize=(20, 8))
plt.subplots_adjust(wspace=0, hspace=.6)

plt.subplot(2,1,1)
plt.xticks(rotation=30)
sns.countplot(x="TeamName_W", data=final_round)

plt.subplot(2,1,2)
plt.xticks(rotation=30)
sns.countplot(x="TeamName_L", data=final_round)

We see(可以看到)：
- Duke won the most Championships, 5 times in total, go Zion. Followed by North Carolina and Connecticut, both 4 times.(冠军数最多的是杜克大学，共获得5次锦标赛冠军，其次是北卡和康涅狄格州，分别获得4次冠军)
- Duke and Michigan were the top runners up, four times, followed by Kansas.(亚军数最多的是杜克和密歇根大学，均为4次，其次是堪萨斯的3次)

### Top 10 in 32,16,8,4,2(进入32,16,8,4,2强次数前10的球队)

In [ ]:
team32 = mSection1.query('DayNum == 136 or DayNum == 137').groupby('TeamName_W')['Season'].count().rename('Count32').reset_index().sort_values(by='Count32',ascending=False).iloc[:10]
team16 = mSection1.query('DayNum == 138 or DayNum == 139').groupby('TeamName_W')['Season'].count().rename('Count16').reset_index().sort_values(by='Count16',ascending=False).iloc[:10]
team8 = mSection1.query('DayNum == 143 or DayNum == 144').groupby('TeamName_W')['Season'].count().rename('Count8').reset_index().sort_values(by='Count8',ascending=False).iloc[:10]
team4 = mSection1.query('DayNum == 145 or DayNum == 146').groupby('TeamName_W')['Season'].count().rename('Count4').reset_index().sort_values(by='Count4',ascending=False).iloc[:10]
team2 = mSection1.query('DayNum == 152').groupby('TeamName_W')['Season'].count().rename('Count2').reset_index().sort_values(by='Count2',ascending=False).iloc[:10]

plt.subplots(figsize=(20, 10))

plt.subplot(2,3,1)
plt.xticks(rotation=30)
sns.barplot(x="TeamName_W", y='Count32', data=team32)
plt.subplot(2,3,2)
plt.xticks(rotation=30)
sns.barplot(x="TeamName_W", y='Count16', data=team16)
plt.subplot(2,3,3)
plt.xticks(rotation=30)
sns.barplot(x="TeamName_W", y='Count8', data=team8)
plt.subplot(2,3,4)
plt.xticks(rotation=30)
sns.barplot(x="TeamName_W", y='Count4', data=team4)
plt.subplot(2,3,5)
plt.xticks(rotation=30)
sns.barplot(x="TeamName_W", y='Count2', data=team2)

we see(可以看到)：
- Kansas had 32 times in the top 32.(进入32强最多的球队是堪萨斯的32次)
- Duke had 25 times in the top 16.(进入16强最多的球队是杜克的25次)
- Kentucky had 17 times in the top 8.进入8强最多的球队是肯塔基的17次；
- Duke had 12 times in the top 4.进入4强最多的球队是杜克的12次；
- Duke had 9 times in the top 2.进入2强最多的球队是杜克的9次；

Combined with the previous analysis, Duke entered the final nine times, and finally won five championships and four runner-up, WOW.(其中结合之前的分析，杜克大学9次进入决赛，最终获得5次冠军，4次亚军的傲人成绩)

### The score gap in every season(各个赛季随时间推进胜负分差的趋势)

In [ ]:
tournament = mSection1.query('DayNum >= 136 and DayNum <= 154')
tournament['ScoreDiv'] = tournament['WScore'] - tournament['LScore']
tournament = tournament.groupby('Season')[['WScore','LScore','ScoreDiv']].mean().reset_index()

plt.subplots(figsize=(20, 5))
sns.barplot(x="Season", y='ScoreDiv', data=tournament)

we see(可以看到)：
- Max gap in 1993 season.(锦标赛平均分差最大的是1993年)
- Min gap in 1985 season.(锦标赛平均分差最小的是1985年)
- Mean gap is almost 11 in last 5 years.(近5年平均分差在11分左右)
- There is little difference between seasons.(各个赛季之间差异不大)

### Seed number and region with champion(冠军对应的种子号统计、区域统计)

In [ ]:
final_round = mSection1.query('DayNum == 154')
final_round['Seed_W_num'] = final_round.Seed_W.apply(lambda seed:int(seed[1:]) if len(seed)==3 else int(seed[1:-1]))
final_round['Seed_L_num'] = final_round.Seed_L.apply(lambda seed:int(seed[1:]) if len(seed)==3 else int(seed[1:-1]))
final_round['Seed_W_code'] = final_round.Seed_W.apply(lambda seed:'Region'+seed[0])
final_round['Seed_L_code'] = final_round.Seed_L.apply(lambda seed:'Region'+seed[0])
final_round['Seed_W_region'] = final_round.apply(lambda row:row[row['Seed_W_code']], axis=1)
final_round['Seed_L_region'] = final_round.apply(lambda row:row[row['Seed_L_code']], axis=1)

plt.subplots(figsize=(20, 10))

plt.subplot(3,1,1)
sns.countplot(x="Seed_W_num", data=final_round)
plt.subplot(3,1,2)
sns.countplot(x="Seed_W_region", data=final_round)
plt.subplot(3,1,3)
sns.lineplot(x="Season", y="Seed_W_num", estimator=None, data=final_round)

we see(可以看到)：
- The No.1 seed of each division is still the hot spot of the champion, No.1 seed had 22 times winning the champion, and the count of Champions is basically linearly negatively related to the seed number, which shows that the seed designation before the competition is quite accurate.(各个分区的1号种子依然是冠军的大热门，共22次获得冠军，且冠军数量与种子号基本线性负相关，这说明赛前的种子指定的还是相当准确的)
- n terms of competition area, Midwest won the most Championships, followed by south.(赛区上看，Midwest获得冠军最多，其次是South，这说明在赛区这一个维度上，实例是不太平衡的，这点跟NBA不太一样，NBA由于存在这工资帽这种机制，使得其各队实力相对会被限制到一个范围内，而大学则是有篮球名校的存在的)
- From the trend chart of the number of seeds of the team that won the championship, it is more and more difficult for the champion to have a complete black horse, which basically appears in the number 1, 2 and 3 seeds.(从获得冠军的球队种子号的趋势图看，冠军越来越难以出现完全的黑马，基本上以1,2,3号种子中出现，当然了，并不是说3号种子就不是黑马)

### Game detail infomation visualization(比赛详细统计数据可视化)

In [ ]:
cols = ['TeamID','Score','FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']
cols = ['W'+col for col in cols] + ['L'+col for col in cols]

regular_data = mSection1.query('is_regular == 1')[cols].mean().rename('value')
tournament_data = mSection1.query('is_regular == 0 and DayNum < 154')[cols].mean().rename('value')
final_data = mSection1.query('DayNum == 154')[cols].mean().rename('value')

tmp = pd.DataFrame({})
tmp = tmp.append([['regular']+list(regular_data.values)],ignore_index=True)
tmp = tmp.append([['tournament']+list(tournament_data.values)],ignore_index=True)
tmp = tmp.append([['final']+list(final_data.values)],ignore_index=True)
tmp.columns = ['match_type']+list(regular_data.index)
tmp

In [ ]:
plt.subplots(figsize=(20, 3*5))

for i,col in enumerate(['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']):
#     plt.subplot(5,3,i*3+1)
#     sns.barplot(x="match_type", y='W'+col, data=tmp)
#     plt.subplot(5,3,i*3+2)
#     sns.barplot(x="match_type", y='L'+col, data=tmp)
#     plt.subplot(5,3,i*3+3)
#     tmp['W'+col+'_L'+col] = tmp['W'+col] - tmp['L'+col]
#     sns.barplot(x="match_type", y='W'+col+'_L'+col, data=tmp)
    plt.subplot(5,3,i+1)
    tmp['W'+col+'_L'+col] = tmp['W'+col] - tmp['L'+col]
    sns.barplot(x="match_type", y='W'+col+'_L'+col, data=tmp)

we see(可以看到)：
- FGM&FGA: There are similar numbers in FGM with both side, but winning team has less FGA.(命中数上看，决赛中球队的差异最小，未命中上看，决赛中，胜利的一方似乎出手更稳重，也就是投丢了更少的球)
- FGM3&FGA3: The winning (三分球上看，在决赛中，胜利的一方甚至进的三分球要少于失败的一方，这也说明一个现象，通常来说决赛因为防守方式、吹罚方式、球员心态等各种因素，一般球队会采取更保守的得分手段，而较少采用不稳定的三分球)
- FTM&FTA: There is little difference in free throws. The winning team has a higher percentage of free throws.(罚球上看差异不大，胜利方的罚球命中率更高)
- Rebounds: On the offensive rebounds, there are always fewer winning teams, especially in the final. Generally we can said, less offensive rebounds can show that the tactics of the team are conservative, that is, to actively retreat from defense, so as to avoid the opportunity of fast attack caused by grabbing offensive rebounds. On the defensive rebounds, the winning side obviously performed better.(篮板球：灌篮高手里面说，赢得篮板的人赢得比赛，看看是不是这样。可以看到对于进攻篮板，胜利的球队总是更少的，决赛尤其如此，通常来说进攻篮板少可以说明球队的战术偏保守，也就是积极退防，以避免因为抢进攻篮板导致对方出现快攻的机会。而防守篮板上看，胜利方明显表现更好)
- Assists and mistakes: in the final, these two are relatively less, which is because the team usually needs to take more personal attacks due to the problems of defensive strength, penalty scale and so on.(助攻和失误：在决赛，这两个相对都更少，这是因为通常决赛由于防守强度、判罚尺度等问题，球队通常需要采取更多的个人进攻，这也是明星球员的作用，而这种方式通常对应的助攻和失误都会少一些)
- Steals and blocks: These are two different defense methods. Generally, steals have certain risk-taking elements. Because the failure of steals usually means loss the position, so there are fewer steals in the final, and more blocks means more attacks to the paint area, which is also caused by more attack methods in the final.(抢断和盖帽：这是两种不同的防守方式，通常抢断是有一定冒险成分的，因为抢断失败通常意味着失位，因此决赛中抢断更少出现，而盖帽多则表示到油漆区的进攻更多，这也是决赛中更多的进攻方式选择导致的)
- In personal fouls, the difference between the winner and the loser in the final is smaller. Of course, it's not very friendly for some players who like to make fouls.(个人犯规上，决赛中胜利方和失败方相比差异更小，一定程度上说也是因为不想把比赛的结果交给裁判，而是更多的给球员发挥，当然这对一些喜欢造犯规的球员来说就不是很友好)

## 数据文件描述(This part only for chinese guy)

数据集中的名称、日期等都做了标准化的处理，对于男女比赛，分别用`M`开头和`W`开头来区分。如果需要引入外部数据的话，可以通过`MTeamSpellings.csv`和`MTeamSpellings.csv`进行名称的映射。

### 数据第一部分 - 基础数据
- 基础信息：
    - 球队ID和球队名。
    - 历史锦标赛种子。
    - 常规赛、竞标赛、NCAA锦标赛的最终分数。
    - 包括日期和地区名称的赛季级的详细信息(关于赛季，比如2020赛季，通常指的是结束于2020年，而不是开始于2020年，因此如果是2019年开始，2020年结束，依然称之为2020赛季)。
- MTeams.csv and WTeams.csv
    - 球队信息，注意不是每个球队在每个赛季都有比赛，因为比赛数据只包含甲级部分。
    - 球队ID：标识一个大学的一只球队，4位数字，1000~1999表示男队，3000~3999表示女队，该ID是不会变的。
    - 球队大学名称的简洁写法。
    - 球队为1级联赛的第一年，最早是1985年，注意这部分只有男队。
    - 球队为1级联赛的最近的一年，如果目前依然是一级，那么该值为2020，同样也只有男队。
- MSeasons.csv and WSeasons.csv
    - 不同赛季的历史数据，包含一些赛季属性。
    - 赛季表示比赛的年份，注意当前是2020赛季。
    - DayZero表示赛季开始的那一天的日期，集合DayNum可以计算当前日期。
    - RegionW,RegionX,RegionY,RegionZ表示四个分组区域，最后四强就是这4个区域中各自的最后冠军。
- MNCAATourneySeeds.csv and WNCAATourneySeeds.csv
    - 锦标赛的种子信息。
    - 赛季年份。
    - 种子：格式为3/4字符组成，前3个如下，例如W01，表示W分区的1号种子，如果有第4位，可能是a/b，进一步区分前3个字符相同的球队。
    - 球队Id。
- MRegularSeasonCompactResults.csv
    - 除NCAA锦标赛外，或者说之前所有比赛信息。
    - 赛季年份。
    - DayNum。
    - 胜利方的Id、分数，失败方的Id、分数。
    - 胜利方是主队、客队，还是在第三方球场。
    - 加时次数，大于等于0.
- MNCAATourneyCompactResults.csv
    - NCAA锦标赛的历史信息，数据与上面一致，注意WLoc全部为N，表示比赛总是在第三方进行。
    - 通过DayNum判断目前锦标赛处于哪一轮：
        1. DayNum=134或135（星期二/星期三）- 最先四场，确定最终的64支球队。
        2. DayNum=136或137（星期四/星期五）- 第一轮，64进32。
        3. DayNum=138或139（周六/周日）- 第二轮，32进16。
        4. DayNum=143或144（星期四/星期五）- 第3轮，也被称为“甜蜜16强”，16进8。
        5. DayNum=145或146（周六/周日）- 第4轮，也被称为“精英八强”或“区域总决赛”，8进4。
        6. DayNum=152（Sat）- 第5轮，也被称为“最终四强”或“全国半决赛”，4进2。
        7. DayNum=154（周一）- 第6轮，也被称为“全国总决赛”或“全国锦标赛”，决出最终的锦标赛冠军。
- MSampleSubmissionStage1.csv
    - 提交文件，第一阶段是过去5年的NCAA每一场比赛做胜负预测，第二阶段则是对今年的锦标赛做预测。
    - ID由SSSS_XXXX_YYYY组成，XXXX和YYYY为两支球队的ID，低位的在前，SSSS为赛季号，这里也看出赛制是一轮制。
    - Pred为XXXX赢得比赛的概率，如果你认为YYYY赢为60%，那这里就应该是40%。
    
### 数据第二部分 - 球队统计得分数据
- 基本信息：
    - 自02-03赛季以来的各种比赛的罚球、防守篮板、失误等数据。
    - 与第1部分的CompactResult相比，前8列是一致的，从WTeamId到NumOT，但是这里会有其他很多数据，比如二分球个数、三分球个数、发球个数、犯规次数等等。
    - 最终分数可以由以下方式表示：2×FGM + FGM3 + FTM，即2×进球数+三分球数+罚球数=总分。
    - 这里的信息与第1部分中的比赛信息是严格对应的。
- MRegularSeasonDetailedResults.csv
    - 03赛季以来所有非NCAA锦标赛以外的比赛的详细信息。
- MNCAATourneyDetailedResults.csv
    - 03赛季以来所有NCAA锦标赛以外的比赛的详细信息。
- 详细指标
    - WFGM：投篮命中数。
    - WFGA：未命中数。
    - WFGM3：3分命中数，注意这部分数据是包含在WFGM中的。
    - WFGA3：3分未命中数。
    - WFTM：罚球命中数。
    - WFTA：罚球未命中数。
    - WOR：进攻篮板。
    - WDR：防守篮板。
    - WAst：助攻数。
    - WTO：失误数。
    - WStl：抢断数。
    - WBlk：盖帽数。
    - WPF：个人犯规数。
    - 失败方为用L替换W，意思一致。

## The end.